<a href="https://colab.research.google.com/github/G-who0212/video_image_processing/blob/main/pytorch/pytorchTutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

#https://www.kaggle.com/code/blurredmachine/lenet-architecture-a-complete-guide/notebook

In [ ]:
# 공개 데이터셋에서 학습 데이터를 내려받습니다.
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# 공개 데이터셋에서 테스트 데이터를 내려받습니다.
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)


  0%|          | 0/26421880 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/29515 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/4422102 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/5148 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



In [ ]:
batch_size = 64

# 데이터로더를 생성합니다.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


In [ ]:
# 학습에 사용할 CPU나 GPU 장치를 얻습니다.
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

# 모델을 정의합니다.
# class NeuralNetwork(nn.Module):
#     def __init__(self):
#         super(NeuralNetwork, self).__init__()
#         self.flatten = nn.Flatten()
#         self.linear_relu_stack = nn.Sequential(
#             nn.Linear(28*28, 512),
#             nn.ReLU(),
#             nn.Linear(512, 512),
#             nn.ReLU(),
#             nn.Linear(512, 10)
#         )

#     def forward(self, x):
#         x = self.flatten(x)
#         logits = self.linear_relu_stack(x)
#         return logits


# class Lenet5(nn.Module):
#     def __init__(self):
#         super().__init__()
#         self.lenet = nn.Sequential(
#             nn.Conv2d(1, 6,(5, 5), padding=2),
#             nn.ReLU(),
#             nn.MaxPool2d((2, 2), stride = 2),
#             nn.ReLU(),
#             nn.Conv2d(6, 16, (5, 5)),
#             nn.ReLU(),
#             nn.MaxPool2d((2, 2), stride = 2),
#             nn.ReLU(),
#             nn.Conv2d(6, 120, (5, 5)),
#             nn.ReLU(),
#             nn.Flatten(),
#             nn.Linear(120, 84),
#             nn.ReLU(),
#             nn.Linear(84, 10)
#         )

#     def forward(self, x):
#         logits = self.lenet(x)
#         return logits

class Lenet5(nn.Module):
    def __init__(self):
        super().__init__()
        self.lenet = nn.Sequential(
            nn.Conv2d(1, 6,(5, 5), padding=2),
            nn.ReLU(),
            nn.MaxPool2d((2, 2), stride = 2),
            nn.ReLU(),
            nn.Conv2d(6, 16, (5, 5)),
            nn.ReLU(),
            nn.MaxPool2d((2, 2), stride = 2),
            nn.ReLU(),
            nn.Conv2d(16, 120, (5, 5)),
            nn.ReLU(),
            nn.Flatten(),
            nn.Linear(120, 84),
            nn.ReLU(),
            nn.Linear(84, 10)
        )  # try adding BatchNorm2d, Dropout

    def forward(self, x):
        logits = self.lenet(x)
        return logits

      

model = Lenet5().to(device)
print(model)

Using cuda device
Lenet5(
  (lenet): Sequential(
    (0): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=(2, 2), stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): ReLU()
    (4): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
    (5): ReLU()
    (6): MaxPool2d(kernel_size=(2, 2), stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): ReLU()
    (8): Conv2d(16, 120, kernel_size=(5, 5), stride=(1, 1))
    (9): ReLU()
    (10): Flatten(start_dim=1, end_dim=-1)
    (11): Linear(in_features=120, out_features=84, bias=True)
    (12): ReLU()
    (13): Linear(in_features=84, out_features=10, bias=True)
  )
)


In [ ]:
loss_fn = nn.CrossEntropyLoss()
#optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3) # try changing SGD to Adam

In [ ]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # 예측 오류 계산
        pred = model(X)
        loss = loss_fn(pred, y)

        # 역전파
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [ ]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [ ]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.290691  [    0/60000]
loss: 0.819354  [ 6400/60000]
loss: 0.534655  [12800/60000]
loss: 0.717670  [19200/60000]
loss: 0.632129  [25600/60000]
loss: 0.564696  [32000/60000]
loss: 0.552877  [38400/60000]
loss: 0.586565  [44800/60000]
loss: 0.586613  [51200/60000]
loss: 0.530917  [57600/60000]
Test Error: 
 Accuracy: 81.8%, Avg loss: 0.489005 

Epoch 2
-------------------------------
loss: 0.352493  [    0/60000]
loss: 0.478632  [ 6400/60000]
loss: 0.302861  [12800/60000]
loss: 0.483355  [19200/60000]
loss: 0.552291  [25600/60000]
loss: 0.490483  [32000/60000]
loss: 0.402842  [38400/60000]
loss: 0.537167  [44800/60000]
loss: 0.510591  [51200/60000]
loss: 0.426997  [57600/60000]
Test Error: 
 Accuracy: 85.7%, Avg loss: 0.398672 

Epoch 3
-------------------------------
loss: 0.271953  [    0/60000]
loss: 0.366555  [ 6400/60000]
loss: 0.208162  [12800/60000]
loss: 0.445169  [19200/60000]
loss: 0.418652  [25600/60000]
loss: 0.455528  [32000/600

In [ ]:
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth
